# 1. Business Understanding

Analyze Acme Co.’s 2014–2018 sales data to identify key revenue and profit drivers across products, channels, and regions; uncover seasonal trends and outliers; and align performance against budgets. Use these insights to optimize pricing, promotions, and market expansion for sustainable growth and reduced concentration risk.

**Objective**  
Deliver actionable insights from Acme Co.’s 2014–2018 sales data to:  

Identify top-performing products, channels, and regions driving revenue and profit  
Uncover seasonal trends and anomalies for optimized planning  
Spot pricing and margin risks from outlier transactions  
Inform pricing, promotion, and market-expansion strategies  
These findings will guide the design of a Power BI dashboard to support strategic decision-making and sustainable growth.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

# df = pd.read_excel(r"C:\Users\hp\Downloads\data.xlsx")    #raw string r
# df = pd.read_excel("C:\\Users\\hp\\Downloads\\data.xlsx") #double backslash as \ is traeted as escape sequence

df is a dictionary of sheets in file. it stores data in {sheet:table}(key value pair) format

In [ ]:
# df = pd.read_excel("C:/Users/hp/Downloads/data.xlsx", sheet_name=None)  #use fwd slash for location
df = pd.read_excel("data.xlsx", sheet_name=None)  #use fwd slash for location
                    #sheetname=none load all sheets


df is a dictionary → {sheet_name: DataFrame}     (sheetnaeme and table in it)  
.items() gives pairs (key, value)  
name = the sheet name (a string)  
table = the DataFrame of that sheet

# 2.Data Understanding

In [ ]:
df.keys()      #print name of all sheets or column names of a specific given sheet

In [ ]:
sales = df["Sales Orders"]      #access one sheet
customer = df["Customers"]    
region = df["Regions"]    
statereg = df["State Regions"]      
product= df["Products"]    
budget = df["2017 Budgets"]    

In [ ]:
# Example: loop through all sheets
for name, table in df.items():
    print(f"\n--- {name} ---")
    print(table.shape)

In [ ]:
sales.head()

In [ ]:
customer.head()

In [ ]:
product.head()

In [ ]:
region.head()

In [ ]:
statereg.head()

In [ ]:
budget.head()

In [ ]:
sales.isnull().sum()

In [ ]:
customer.isnull().sum()

In [ ]:
region.isnull().sum()

In [ ]:
statereg.isnull().sum()

In [ ]:
budget.isnull().sum()

In [ ]:
product.isnull().sum()

# 3.Data Connection

In [ ]:
dfnew=df.copy()

In [ ]:
dfnew= pd.merge(sales,customer, left_on="Customer Name Index", right_on="Customer Index", how="left")

In [ ]:
# sales.merge(customer,left_on="Customer Name Index", right_on="Customer Index" )

when column names is not same in both tables, left_on = col_name in table1 and right_on= col_name in table2 is used  
if col name is same "on" is used  
merge(t1,t2, on="id")

In [ ]:
dfnew= dfnew.merge(product, left_on="Product Description Index", right_on="Index", how="left")

In [ ]:
dfnew=dfnew.merge(region, left_on="Delivery Region Index", right_on="id", how="left")

In [ ]:
dfnew=dfnew.merge(statereg, left_on="state_code", right_on="State Code", how="left")

In [ ]:
dfnew=dfnew.merge(budget, left_on="Product Name", right_on="Product Name", how="left")

**Merged file creation**

In [ ]:
# dfnew.to_csv("salesfile.csv")

# 4. Data cleaning

In [ ]:
dfnew.drop(columns=["Customer Index","Index","id","State Code","State"], errors="ignore", inplace=True)
# df=df.drop(columns=["Customer Index","Index","id","State Code","State"], errors="ignore")     assigning directly without inplace

In [ ]:
dfnew.keys()

**convert column names to lower case for easeof access**

In [ ]:
dfnew.columns=dfnew.columns.str.lower()
dfnew.columns.values

**recognise which data is needed and keep it**

In [ ]:
col_to_keep=['ordernumber', 'orderdate','channel','order quantity', 'unit price',
       'line total', 'total unit cost','customer names', 'product name','county','state', "state_code",'region',"latitude","longitude",'2017 budgets']
dfnew=dfnew[col_to_keep]

**rename columns**

In [ ]:
dfnew=dfnew.rename(columns={
    'ordernumber':'order_number',
    'orderdate':'order_date',  
    'order quantity':'order_quantity',
    'unit price':'unit_price',
    'line total':  'revenue', 
    'total unit cost':'total_unit_cost', 
    'customer names':'customer_name', 
    'product name':'product_name',
    '2017 budgets':'budget_2024'}
    )
dfnew.keys()

In [ ]:
dfnew=dfnew.rename(columns={"line_total":"revenue"})
dfnew.keys()

#moving the years to make data look recent

method 1:  
df.loc[df["order_date"].dt.year == 2014, "order_date"] = df["order_date"] + pd.offsets.DateOffset(years=7)  

method 2:  
df["order_date"]= df["order_date"].apply(
    lambda d: d.replace(year=2021) if d.year==2014 else
            d.replace(year=2022) if d.year == 2015 else 
            d.replace(year=2023) if d.year == 2016 else 
            d.replace(year=2024) if d.year == 2017 else 
            d.replace(year=2025) 
)  

method 3:  


In [ ]:
year_map = {2014: 2021, 2015: 2022, 2016: 2023, 2017: 2024, 2018:2025}

# Extract components
years  = dfnew["order_date"].dt.year.replace(year_map)
months = dfnew["order_date"].dt.month
days   = dfnew["order_date"].dt.day

# Rebuild dates safely (clip avoids 29 → 28)
dfnew["order_date"] = pd.to_datetime({
    "year": years,
    "month": months,
    "day": days.clip(upper=28)    #leap year 29feb is changed to 28 feb
})


pd.to_datetime() → converts a set of year/month/day numbers into full datetime objects.

as budget is only available for 2024, blank out budget for all other years

In [ ]:
dfnew.head()

In [ ]:
dfnew.keys()

In [ ]:
dfnew.loc[dfnew["order_date"].dt.year !=2024, "budget_2024"] = pd.NA

In [ ]:
dfnew.to_csv("final.csv")

In [ ]:
dfnew.info()

filter dataset to include only record from 2017

In [ ]:
df_2017= dfnew.loc[dfnew["order_date"].dt.year ==2021, :]

In [ ]:
dfnew.isnull().sum()

# 5.Feature engineering

In [ ]:
dfnew["profit"]= dfnew["revenue"]- dfnew["total_unit_cost"]        #adding new columns
dfnew["profit_margin_pct"]= (dfnew["profit"]/dfnew["revenue"])*100

In [ ]:
dfnew.keys()

# 6. EDA

creating monthly sales line chart

In [ ]:
# Ensure order_date is datetime
dfnew['order_date'] = pd.to_datetime(dfnew['order_date'])
#
# Extract month and year for grouping
dfnew['year_month'] = dfnew['order_date'].dt.to_period('M')   #add year automatically to period
dfnew['year_month'].head()


In [ ]:
# Aggregate total revenue per month
# df.groupby(by)[column].agg_function()
#reset_index coverts it into dataframe not talking period as index
monthly_sales = dfnew.groupby('year_month')['revenue'].sum().reset_index()
# 

In [ ]:
monthly_sales['year_month'] = monthly_sales['year_month'].dt.to_timestamp()  # convert to datetime for plotting
# #as matlplotlib work better with timestamps convert period y-m to timestsamp y-m-d
# monthly_sales_2023=monthly_sales.loc[monthly_sales["year_month"].dt.year==2023, :]  #gives sales for 2023

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(monthly_sales['year_month'], monthly_sales['revenue'], marker='o', linestyle='-')
plt.title('Monthly Sales')
plt.xlabel('Month')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()
